In [ ]:
import numpy as np

import pandas as pd

In [ ]:
from bs4 import BeautifulSoup
import requests
def url(id):
    return f"https://www.imdb.com/title/tt{id:07d}/"
def get(id):
    try:
        u = url(id)
        resp = requests.get(u,headers={
            "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
        })
        if resp.status_code != 200:
            # print(resp.content.decode())
            return (id,-1)
        soup = BeautifulSoup(resp.content.decode())
        [div] = soup.find_all("div", {
            "class": "sc-491663c0-7 dUfBfF"
        })
        [img] = div.findAll("img")
        return img["src"]
    except Exception as e:
        return (id,-1)
        raise e
    return (id,-1)
url(114709)

In [ ]:
from filldb import read_df

In [ ]:
ratings = read_df("kek")


In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
def get_imdbid(row):
    return int(row.imdbId)

async def get_work_done():
    with ThreadPoolExecutor(max_workers=10) as executor:
        loop = asyncio.get_event_loop()
        tasks = [
            loop.run_in_executor(
                executor,
                get,
                *(get_imdbid(row))  # Allows us to pass in arguments to `lyrics_from_year`
            )
            for i, row in ratings.iloc[:10].iterrows()
        ]

    return await asyncio.gather(*tasks)

loop = asyncio.get_event_loop()
future = asyncio.ensure_future(get_work_done())
loop.run_until_complete(future)
# final_lyric_series = pd.Series(name="lyrics")
# for result in future:
#     final_lyric_series = pd.concat([final_lyric_series, result])
#     print(result)

In [ ]:
import pandas as pd
import torch
from torch import nn,optim
import torch.nn.functional as F
import torch.utils.data
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
import matplotlib.pyplot as plt
import tqdm

In [ ]:
a = torch.rand((10,))
vals,ids = a.sort()
ids

In [ ]:
def plot(losses, stds, f1):
    ax1, ax2, ax3 = axes = plt.subplots(1, 3, figsize=(10, 3))[1]
    for ax, title, arr in zip(axes, ["Loss", "F1", "Std"], [losses, f1, stds]):
        ax.plot(arr, label=title)
        ax.set_ylim([0, None])
        ax.legend()
    ax2.set_ylim([0, 1])
    plt.plot()

In [ ]:
def predict(model,ratings):
    userIds = [0,1,2,3]
    movieIds = [1,30,177,10]
    _userIds = torch.repeat_interleave(torch.tensor(userIds), len(movieIds)).tolist()

    pred,_,_ = model.forward(torch.tensor(userIds),torch.tensor(movieIds))
    pred = (pred*2).round()/2
    df = pd.DataFrame({"userId":_userIds ,"movieId": movieIds*len(userIds), "pred_rating": pred.flatten().tolist()})

    df = df.merge(ratings.loc[ratings.userId.isin(userIds)].loc[ratings.movieId.isin(movieIds)][["movieId","rating","userId"]],
            on=["userId","movieId"],how="outer")
    return df

In [ ]:
from filldb import read_df
ratings = read_df("")
total_ratings = ratings[["userId", "imdbId"]].groupby("imdbId").count().reset_index().rename(columns={"userId": "total_ratings", "imdbId": "imdbId"},)
ratings = ratings.merge(total_ratings, on="imdbId").reset_index(drop=True)
ratings

In [ ]:
ratings[["userId","movieId","rating"]].groupby("movieId").mean()